In [1]:
import spacy
nlp=spacy.load('en')
import PyPDF2
import subprocess
import csv
import docx2txt
import re
import os
import json
import re
import operator
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymongo import MongoClient
from pprint import pprint
from gensim import corpora
from collections import defaultdict
from gensim import corpora, models, similarities
from gensim.models import Word2Vec
model = Word2Vec.load("Resume101")  

section_words = [
     'abilities', 'academic', 'accomplishments', 'achievement', 'activities', 
    'additional', 'attributes', 'award', 'background', 'brief', 'career', 
    'certification', 'chronicle', 'competencies', 'credential',
    'curricular', 'declaration', 'detail', 'dossier', 'education', 'educational',
    'employment', 'executive', 'experience', 'experience', 'exposure', 'extra',
    'goals', 'highlights', 'history', 'hobby', 'industry', 'information', 
    'interests', 'internship', 'involved', 'it', 'job', 'objective', 
    'organisation', 'organization', 'overview', 'papers', 'particulars', 'path', 
    'publication', 'personal', 'presentation', 'professional', 'proficiency', 'profile', 
    'programming', 'project', 'published', 'qualification', 'recognition', 'details',
    'related', 'scholastic', 'set', 'skill', 'snapshot', 'summary', 'synopsis', 'technical' , 
    'traits', 'training', 'undertaken', 'work','academia','education:-','interests'
]
sections=['objective','summary','project','experience','skill','education','achievement']

In [2]:
def textify(f):
    if f.endswith(".doc"):
        subprocess.call(['abiword', '--to=docx', f])
        subprocess.call(['rm', f])
        f = f.replace('.doc','.docx')
    if f.endswith(".docx"):
        text = docx2txt.process(f)
        parsedData = nlp(text)
        return parsedData
    elif f.endswith(".pdf"):
        pdfObj = open(f,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfObj)
        numpages = pdfReader.getNumPages()
        fileObj = ''
        for i in range(numpages):
            pageObj = pdfReader.getPage(i)
            fileObj += pageObj.extractText()
        parsedData = nlp(fileObj)
        return parsedData
    elif f.endswith(".txt"):
        with open (f, "r") as myfile:
            data=myfile.read()
        pd=nlp(data)
        return pd
    else:
        strin = "none"
        parsedData = nlp(strin)
        return parsedData

def headings(text):
    for line_no,line in enumerate(text.split('\n')):
        if(line!=''):
            for word in nlp(line):

                if word.lemma_ in section_words:
                    line=line.strip()
                    #print(word.lemma_,len(line.split(" ")))
                    if(len(line.split(" "))<=5):
                        print(line.upper().strip())

                    break

def Personal_Info(pd,sec):
    if not pd:
            return
    pattern = re.compile("resume", re.IGNORECASE)
    pd=pattern.sub("", pd.text)
    pattern = re.compile("curriculum vitae", re.IGNORECASE)
    pd=pattern.sub("", pd)
    name=""
    pd=nlp(pd)
    for ent in pd.ents:
        if ent.label_=="PERSON":
            name=ent.text
        break
    if name=="":
        name=pd[0:2].text
    sec['name']=name.strip()
    mobile = ""
    match_mobile = re.search(r'((?:\(?\+91\)?)?\d{10})',pd.text)
    if(match_mobile != None):
        mobile = match_mobile.group(0)
    sec['phone']=mobile
    email = ""
    match_mail = re.search(r'[\w\.-]+@[\w\.-]+', pd.text)
    if(match_mail != None):
        email = match_mail.group(0)
    sec['emailid']=email

def cleaner(pd):
    li = []
    pd = pd.replace("     ","")
    pd = pd.replace("   ","")
    pd = pd.replace("  ","")
    pd = pd.replace("\n\n\n\n\n"," ")
    pd = pd.replace("\n\n\n"," ")
    pd = pd.replace("-"," ")
    i = 0
    while i < len(pd):
        st = ""
        while(pd[i] != '\n' and i < len(pd)-1):
            st = st + pd[i]
            i += 1
        st = st + pd[i]
        if(st != "" and st != " " and st != "\n"):
            st = st.strip("\t")
            st = st.strip(" ")
            st = st.strip("\n")
            st = st.replace("\t", " ")
            li.append(st)
        i += 1
    return li

def sentence_tokenizer(texts,sec): 
    prev_section_lineno=0;
    lines=texts.split('\n')
    section_no=0
    heading_cleaned=[]
    for line_no,line in enumerate(texts.split('\n')):

        if(line!=''):
            line=line.strip()    

            if(len(line.split(" "))<5):
                #print(line)
                for word in nlp(line):

                    if word.lemma_ in section_words:   

                        if(section_no==0):
                            prev_section_lineno=line_no
                        else :                         
                            section_text=lines[prev_section_lineno+1:line_no]
                            section_text=list(map(str.strip,section_text))
                            section_text=list(filter(None, section_text))
                            flag_section=0

                            for word in heading_cleaned:
                                if word in sections:
                                    sec[word].extend(list(section_text))
                                    flag_section=1
                                    break

                                elif word =='career':
                                    sec['experience'].extend(list(section_text))

                                elif word in ['hobby','declaration','personal'] :
                                    flag_section=1

                            #When headings can't be directly matched, word2vec model Resume101 is used to
                            #obtain the closest match for the heading

                            if(flag_section==0):
                                stops = set(stopwords.words("english"))
                                heading_cleaned = [w for w in heading_cleaned if not w in stops]
                                heading_cleaned=list(filter(None, heading_cleaned))
                                education_match=0
                                skills_match=0
                                summary_match=0
                                work_match=0
                                achievement_match=0

                                for word in heading_cleaned:
                                    try:
                                        education_match+=model.similarity(word,'education')
                                        skills_match+=model.similarity(word,'skill')
                                        summary_match+=model.similarity(word,'summary')
                                        work_match+=model.similarity(word,'work')
                                        achievement_match+=model.similarity(word,'achievement')
                                    except KeyError:
                                        heading_cleaned.remove(word)


                                if(heading_cleaned):
                                    education_match/=len(heading_cleaned)
                                    skills_match/=len(heading_cleaned)
                                    summary_match/=len(heading_cleaned)
                                    work_match/=len(heading_cleaned)
                                    achievement_match/=len(heading_cleaned)

#                                 print("Skill match",skills_match)
#                                 print("achievement match",achievement_match)
                                if(education_match>=0.58):
                                    sec['education'].extend(section_text)

                                if(skills_match>=0.60):
                                    sec['skill'].extend(section_text)

                                if(summary_match>=0.62):
                                    sec['summary'].extend(section_text)

                                if(work_match>=0.60):
                                    sec['experience'].extend(section_text)

                                if( achievement_match>=0.66):
                                    sec['achievement'].extend(section_text)



                            prev_section_lineno=line_no  

                        heading_cleaned=clean_headings(line)
                        section_no+=1
                        break



    section_text=lines[prev_section_lineno+1:line_no]

    for word in heading_cleaned:

        if word in sections:

            sec[word]=section_text

    return sec

#Function to convert a heading into cleaned words

def clean_headings(heading):

    #Remove punctuations
    heading= re.sub("[^a-zA-Z0-9+]"," ", heading)
    #Removes newlines
    heading=heading.strip()
    #Convert to lowercase
    heading=heading.lower()
    line=nlp(heading)
    #lemmatize the words and return
    heading_lemma=[word.lemma_ for word in line]
    return heading_lemma    

def sectionize(file,sec):
    text=textify(file)
    Personal_Info(text,sec)
    text=text.string
    sentence_tokenizer(text,sec)

def entitiser(st):
    pd = nlp(st)
    st1 = ""
    pd = pd.string
    pd = pd.replace("     ","")
    pd = pd.replace("   ","")
    pd = pd.replace("  ","")
    pd = pd.replace("\n\n\n\n\n","")
    pd = pd.replace("\n\n\n","")
    pd = pd.replace("(","")
    pd = pd.replace(")","")
    pd = pd.replace("&","")
    pd = pd.replace(":","")
    i = 0
    while i < len(pd):
        st = ""
        while(pd[i] != '\n' and i < len(pd)-1):
            st = st + pd[i]
            i += 1
        st = st + pd[i]
        if(st != "" and st != " " and st != "\n"):
            st = st.strip("\t")
            st = st.strip(" ")
            st = st.strip("\n")
            st = st.replace("\t", " ")
#             print(st)
            st1 += st
            st1 += " "
#             print("------------------------------------")
        i += 1
    !cd '/home/tarun/Downloads/stanford-ner-2016-10-31' ; echo $st1 > entity_text.txt
    !cd '/home/tarun/Downloads/stanford-ner-2016-10-31' ; java -mx600m -cp "*:lib/*" edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier '/home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz' -textFile entity_text.txt > out.txt
    l = !cd '/home/tarun/Downloads/stanford-ner-2016-10-31'; cat out.txt
    li = []
    for i in range(len(l)):
        li = li + l[i].split(" ")
    for i in range(len(li)):
        li[i] = li[i].replace("/", " /")
    return li

def insert_db(resume_json):
    client = MongoClient()
    mydb = client['project2']
    profiles = mydb.profiles
    profiles.insert(resume_json)
    return

def profiles_builder():
    count = 1
    path="/home/tarun/Dropbox/Final year Project/Final/Dataset/Resumes"

    for (dirpath, dirnames, filenames) in os.walk(path):
        for file in filenames:
            file = os.path.join(dirpath, file)
            sec={'name':'','phone':'','emailid':'','objective':[],'summary':[],'project':[],'experience':[],'skill':[],'education':[],'achievement':[]}

            sectionize(file,sec)
            print(sec)

            #PROJECT
            proj=' '.join(sec['project'])
            proj=cleaner(proj)

            #EXPERIENCE
            st=' '.join(sec['experience'])
            li = entitiser(st)                      
            list_skills=[]
            list_exp=[]
            for i in range(len(li)):
                if ("/O" not in li[i] and li[i] != "") or "/ORG" in li[i]:
                    list_exp.append(li[i])
                if ("/SKILL" in li[i] and "," not in li[i]):
                    s1 = li[i]
                    s1 = s1.replace(" /SKILL","")
                    list_skills.append(s1)

            #SKILLS
            st = ""
            for i in range(len(sec['skill'])):
                st += sec['skill'][i]
                st += " "
            skill_text = ""
            li = cleaner(st)
            if li and li[0] != "":
                skill_text = li[0] + " "
            if proj and proj[0] != "":
                skill_text = proj[0] + " "
            li = entitiser(skill_text)
            for i in range(len(li)):
                if ("/SKILL" in li[i] and "," not in li[i]):
                    s1 = li[i]
                    s1 = s1.replace(" /SKILL","")
                    list_skills.append(s1)      
            ski=','.join(sec['skill'])
            ski=cleaner(ski)
            ski=','.join(ski)
            ski =ski.split(",")
            #print(ski)
            for i in range(len(ski)):
                ski[i] = ski[i].strip(" ")
            list_skills = list_skills + ski
            list_skills = set(list_skills)
            list_skills = list(list_skills)

            resume_data={'prof_id':'','name':'','phone':'','email':'','projects':[],'experience':[],'skills':[],'objective':[],'keywords':[]}
            resume_data['prof_id']= count
            count += 1
            print("-------------------------------------------",count,"-----------------------------------------")
            resume_data['name']=sec['name'].title()
            resume_data['phone']=sec['phone']
            resume_data['email']=sec['emailid']
            resume_data['projects']=proj
            resume_data['experience']=list_exp           
            resume_data['skills']= list_skills
            resume_data['objective']=sec['objective']
            #resume_data['keywords']=keys
#             for key, value in resume_data.items() :
#                 print (key.upper(), value)
#                 print('-----------------------------------------')

            r = json.dumps(resume_data)
            resume_json = json.loads(r)
            insert_db(resume_json)

# Single file parsing

# Main function

In [3]:
client = MongoClient()
db = client.project2
db.profiles.count()

101

In [9]:
profiles_builder()

{'education': ['Completed Graduation BBA from CCS University  ( 2010 )', '12th Passed from U.P. Board    ( 2006 )', '10th Passed from U.P. Board    ( 2004 )', 'Web  &  Multimedia Animation course from OXFORD Institute, Delhi. ( 2011 )', 'Careot.net', 'Highadventuretour.com', 'My Online Portfolio', 'www.webdesizner.in', 'Completed Mobile App Projects', 'ConnectTo   ( android )', 'Careot App   (android,  iphone )', 'ShineTrack  ( android )', 'Mother Valley School App  ( android )', 'Juju - Mobile Dialer  ( android, iphone )', 'Platinumfone - Mobile Dialer ( android )', 'Guru-edu School App ( android )', 'Golf  Fundraiser  ( iphone )', 'Madeena786 - Mobile Dialer ( android )', 'Kidsave-golf foundraiser app ( iphone )', 'Bethesda curry kitchen app ( iphone )', 'Currently Working On Mobile App Projects', 'ConnectU   ( android )'], 'skill': ['Photoshop', 'Corel Draw', 'Dreamweaver', 'Visual Studio 2012', 'HTML5', 'CSS3', 'Bootstrap', 'Java Script', 'jquery'], 'emailid': 'surajget.singh@gmail

/home/tarun/miniconda3/envs/spacy/lib/python3.5/site-packages/ipykernel/__main__.py:263: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


Invoked on Tue Jun 27 20:46:07 IST 2017 with arguments: -loadClassifier /home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz -textFile entity_text.txt
loadClassifier=/home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz
textFile=entity_text.txt
Loading classifier from /home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz ... done [0.5 sec].
CRFClassifier tagged 261 words in 9 documents at 1345.36 words per second.
Invoked on Tue Jun 27 20:46:08 IST 2017 with arguments: -loadClassifier /home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz -textFile entity_text.txt
loadClassifier=/home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz
textFile=entity_text.txt
Loading classifier from /home/tarun/Downloads/stanford-ner-2016-10-31/NER Training/Experience-model.ser.gz ... done [0.4 sec].
CRFClassifier tagged 137 words in 3 documents at 878.21 words p

# Mongo check

In [4]:
client = MongoClient()
db = client.project2
db.profiles.count()

101

In [ ]:
pprint(db.profiles.find_one({"prof_id": 35}))

# Generating documents

In [8]:
documents = []
client = MongoClient()
db = client.project2
for profile in db.profiles.find():
    li = (profile["skills"])
#     print(li)
    li = ' '.join(li)
#     print(li)
    if li =="":
        documents.append("None")
    else:
        documents.append(li)
#     print("------------------------------------")

# Matching- Indexing by LSA

In [9]:
len(documents)

101

In [10]:
stoplist = set('for a of the and to in : ,'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]
# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
dictionary = corpora.Dictionary(texts)
dictionary.save('/home/tarun/Dropbox/Final year Project/Tarun/Matching/temp.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(804 unique tokens: ['angular', 'junit.', 'monitoring', 'time', 'coreldraw']...)


# JD input

In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/home/tarun/Dropbox/Final year Project/Tarun/Matching/temp.mm', corpus)  # store to disk, for later use
print(corpus)

[[(0, 1), (1, 2), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(2, 2), (3, 1), (8, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 3), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 4), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 6), (38, 1), (39, 1), (40, 3), (41, 1), (42, 1), (43, 2), (44, 1), (45, 3), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 3), (52, 1), (53, 1), (54, 2), (55, 3), (56, 3), (57, 2), (58, 1), (59, 1), (60, 4), (61, 1), (62, 1)], [(3, 2), (19, 2), (24, 1), (33, 1), (48, 1), (49, 1), (54, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 2), (76, 2), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1)], [(1, 1), (2, 1), (3, 5), (6, 1), (7, 1), (8, 1), (11, 2), (16, 1), (19, 4), (21, 1), (24, 1), (31, 1), (32, 1), (33, 1), (37, 2), (39, 1), (45,

In [12]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [14]:
#doc = "Web Developer, HTML, Front End Developer, CSS, Bootstrap, Javascript, UI"
doc = "Java Script, Photoshop, Visual Studio, 2012, HTML5, Corel Draw, Java, Dreamweaver, jquery, Bootstrap, Script, CSS3"

In [15]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.20612977761364756), (1, -0.34638368681167381)]


# Ranking Algorithm

In [16]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [17]:
index.save('/home/tarun/Dropbox/Final year Project/Tarun/Matching/temp.index')
index = similarities.MatrixSimilarity.load('/home/tarun/Dropbox/Final year Project/Tarun/Matching/temp.index')

In [18]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.99975431), (1, 0.99878961), (2, 0.98888552), (3, 0.98108077), (4, 0.77007782), (5, 0.99875867), (6, 0.38183442), (7, 0.98858845), (8, 0.67159069), (9, 0.99442327), (10, 0.98949683), (11, 0.99890053), (12, 0.76684988), (13, 0.86189473), (14, 0.94891989), (15, 0.99834526), (16, 0.99695182), (17, 0.96139795), (18, 0.0), (19, 0.50116098), (20, 0.99396491), (21, 0.96027082), (22, 0.99057519), (23, 0.99996817), (24, 0.95114255), (25, 0.27795756), (26, -0.11530036), (27, 0.87563932), (28, 0.98008931), (29, 0.74095702), (30, 0.98730904), (31, 0.99248379), (32, 0.1192331), (33, 0.99961066), (34, 0.99782038), (35, 0.9984417), (36, 0.99186558), (37, 0.92150539), (38, 0.53084779), (39, 0.99632871), (40, 0.9791199), (41, 0.9877336), (42, 0.98919553), (43, 0.9982959), (44, 0.99764514), (45, 0.88258374), (46, 0.99477136), (47, 0.99799329), (48, 0.99959946), (49, 0.94335735), (50, 0.99469888), (51, 0.83154303), (52, 0.048275262), (53, 0.89797628), (54, 0.99951839), (55, 0.0), (56, 0.99793494), 

In [19]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(23, 0.99996817), (64, 0.99984241), (0, 0.99975431), (33, 0.99961066), (48, 0.99959946), (54, 0.99951839), (62, 0.99937326), (11, 0.99890053), (1, 0.99878961), (5, 0.99875867), (35, 0.9984417), (15, 0.99834526), (43, 0.9982959), (47, 0.99799329), (56, 0.99793494), (34, 0.99782038), (44, 0.99764514), (72, 0.99741924), (97, 0.99729252), (16, 0.99695182), (39, 0.99632871), (75, 0.99616069), (46, 0.99477136), (50, 0.99469888), (9, 0.99442327), (60, 0.99415946), (20, 0.99396491), (81, 0.99350595), (31, 0.99248379), (36, 0.99186558), (58, 0.9906497), (22, 0.99057519), (83, 0.9898026), (10, 0.98949683), (42, 0.98919553), (84, 0.9889617), (2, 0.98888552), (7, 0.98858845), (59, 0.98857176), (41, 0.9877336), (30, 0.98730904), (88, 0.98661834), (86, 0.98499918), (69, 0.98487723), (3, 0.98108077), (68, 0.98093557), (28, 0.98008931), (40, 0.9791199), (57, 0.97507799), (74, 0.97270095), (91, 0.96620947), (17, 0.96139795), (21, 0.96027082), (99, 0.95634627), (94, 0.95430177), (95, 0.9542495), (61, 0

In [35]:
client = MongoClient()
db = client.project1
pprint(db.profiles.find_one({"prof_id": 34}))

{'_id': ObjectId('59526d76a8df4709f7d344d0'),
 'email': 'tanv1shrimal@gmail.com',
 'experience': [],
 'keywords': [],
 'name': 'Tanvi Shrimal',
 'objective': [],
 'phone': '9713490838',
 'prof_id': 34,
 'projects': ['Project Name  : MobikeBazar URL : '
              'http://projects.lollypop.biz/mobikebazar/ Skills Used : HTML5, '
              'CSS3, JavaScript, jQuery Team Size : 2 Brief Description  :  '
              'Mobikebazar is the one stop shop for all your spare parts and '
              'accessories needs as long as they have anything to do with two '
              'wheelers. In order to serve our clients, we rely on our decades '
              'of experience and now, our hard work and dedication has made us '
              'the best in the business. (This site is still under '
              'development) Project Name  : Ybann URL : http://www.ybann.com/ '
              'Skills Used : HTML5, CSS3, JavaScript, jQuery, PHP, AngularJS '
              'Team Size : 2 Brief Descr

In [24]:
for i in range(2,12):
    j = sims[i][0]
    client = MongoClient()
    db = client.project1
    prof = db.profiles.find_one({"prof_id": j+1})
    print("Rank:", i-1, end = " ---- ")
    print("Profile id:",j+1," --- ", "Similarity: ", (sims[i][1])*100)
    print("--------------------------------------")

Rank: 1 ---- Profile id: 1  ---  Similarity:  99.9754309654
--------------------------------------
Rank: 2 ---- Profile id: 34  ---  Similarity:  99.961066246
--------------------------------------
Rank: 3 ---- Profile id: 49  ---  Similarity:  99.9599456787
--------------------------------------
Rank: 4 ---- Profile id: 55  ---  Similarity:  99.951839447
--------------------------------------
Rank: 5 ---- Profile id: 63  ---  Similarity:  99.937325716
--------------------------------------
Rank: 6 ---- Profile id: 12  ---  Similarity:  99.8900532722
--------------------------------------
Rank: 7 ---- Profile id: 2  ---  Similarity:  99.8789608479
--------------------------------------
Rank: 8 ---- Profile id: 6  ---  Similarity:  99.8758673668
--------------------------------------
Rank: 9 ---- Profile id: 36  ---  Similarity:  99.8441696167
--------------------------------------
Rank: 10 ---- Profile id: 16  ---  Similarity:  99.8345255852
--------------------------------------


In [26]:
client = MongoClient()
db = client.project1
pprint(db.profiles.find_one({"prof_id": 1}))

{'_id': ObjectId('59526d20a8df4709f7d3448e'),
 'email': 'surajget.singh@gmail.com',
 'experience': ['Guru /ORG',
                'E. /ORG',
                'Technology /ORG',
                '` /DES',
                'Web /DES',
                'Designer /DES',
                '2012 /DATE'],
 'keywords': [],
 'name': 'Surendra',
 'objective': ['To work diligently with an organization that offers '
               'challenging career & which rewards hard work sincerity & '
               'performance.',
               'About Me',
               'Over 4+ years of experience in IT industry working for various '
               'domains like E-commerce, Mobility, IT etc and hands on with '
               'various technologies like HTML 5, CSS 3, Jquery, Bootstrap, '
               'CSS, Javascript etc.'],
 'phone': '8826250144',
 'prof_id': 1,
 'projects': [],
 'skills': ['Corel Draw',
            'jquery',
            'HTML5',
            'CSS3',
            'Dreamweaver',
            'Phot